# Demo 1 - similarities on a sphere

In [ ]:
!pip install --quiet "astrapy>=1.0.0" "python-dotenv>=1.0.0"

In [ ]:
import getpass
import math
import os

from dotenv import load_dotenv

from astrapy import DataAPIClient
from astrapy.constants import VectorMetric

## Setup DB

In [ ]:
load_dotenv()

if "ASTRA_DB_APPLICATION_TOKEN" not in os.environ:
    os["ASTRA_DB_APPLICATION_TOKEN"] = getpass.getpass("Please input your Astra DB Token:")

if "ASTRA_DB_API_ENDPOINT" not in os.environ:
    os.environ["ASTRA_DB_API_ENDPOINT"] = input("Please input your Astra DB API Endpoint:")

if "ASTRA_DB_KEYSPACE" not in os.environ:
    _namespace = input("(Optional) Input your Astra DB namespace if desired, or leave blank:")
    if _namespace:
        os.environ["ASTRA_DB_KEYSPACE"] = _namespace

ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]
ASTRA_DB_API_ENDPOINT = os.environ["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_KEYSPACE = os.environ.get("ASTRA_DB_KEYSPACE")

In [ ]:
db = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN).get_database_by_api_endpoint(ASTRA_DB_API_ENDPOINT, namespace=ASTRA_DB_KEYSPACE)

## Tools

In [ ]:
def vector_norm(v):
    return (sum(v_i*v_i for v_i in v))**0.5

print("Norm test:")
print(f"  [3, 6, 2] => {vector_norm([3, 6, 2]):.4f}")
print(f"  [0.9877, 0.1564] => {vector_norm([0.9877, 0.1564]):.4f}")

In [ ]:
v_documents = [
    {"name": "Evarcha",      "$vector": [0.4045, 0.7939, 0.4540]},
    {"name": "Marpissa",     "$vector": [0.1106, 0.6984, 0.7071]},
    {"name": "Salticus",     "$vector": [0.3673, 0.2668, 0.8910]},
    {"name": "Heliophanus",  "$vector": [0.4156, 0.5721, 0.7071]},
    {"name": "Tmarus",       "$vector": [0.9045, 0.2939, 0.3090]},
    {"name": "Amaurobius",   "$vector": [0.9877, 0.1564, 0.0000]},
    {"name": "Enoplognatha", "$vector": [-0.0483, -0.1488, 0.9877]},
]

#### Sanity check: vector norms

(we truncated the component for the sake of clarity, so this will be approximated to the 4th digit or so...)

In [ ]:
print("Document norms:")
for v_document in v_documents:
    norm = vector_norm(v_document["$vector"])
    print(f"  Norm for {v_document['name']}: {norm:0.4f}")

#### Curious as to how we created these vectors?

The answer is trigonometry and the [polar coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system#Cartesian_coordinates) for the 3D sphere:

```python
def sphere_vector(theta, phi, norm=1):
    rz = norm * math.sin(theta)
    rx = norm * math.cos(theta) * math.sin(phi)
    ry = norm * math.cos(theta) * math.cos(phi)
    return [rx, ry, rz]


v_documents = [
    {"name": "Evarcha", "$vector": sphere_vector(math.pi * 0.15, math.pi * 0.15)},
    {"name": "Marpissa", "$vector": sphere_vector(math.pi * 0.25, math.pi * 0.05)},
    {"name": "Salticus", "$vector": sphere_vector(math.pi * 0.35, math.pi * 0.30)},
    {"name": "Heliophanus", "$vector": sphere_vector(math.pi * 0.25, math.pi * 0.20)},
    {"name": "Tmarus", "$vector": sphere_vector(math.pi * 0.10, math.pi * 0.40)},
    {"name": "Amaurobius", "$vector": sphere_vector(math.pi * 0.00, math.pi * 0.45)},
    {"name": "Enoplognatha", "$vector": sphere_vector(math.pi * 0.55, math.pi * 0.10)},
]

query = sphere_vector(math.pi * 0.18, math.pi * 0.32)
```

## Euclidean ANN

In [ ]:
collection_e = db.create_collection(
    "craftdemo3d_euclidean",
    dimension=3,
    metric=VectorMetric.EUCLIDEAN,
    check_exists=False,
)

In [ ]:
collection_e.insert_many(v_documents)

In [ ]:
query = [0.7129, 0.4524, 0.5358]

for document in collection_e.find(vector=query):
    print(document["name"])

## Cosine ANN

In [ ]:
collection_c = db.create_collection(
    "craftdemo3d_cosine",
    dimension=3,
    metric=VectorMetric.COSINE,
    check_exists=False,
)

In [ ]:
collection_c.insert_many(v_documents)

In [ ]:
for document in collection_c.find(vector=query):
    print(document["name"])

In [ ]:
for document in collection_c.find(vector=query):
    print(document["name"][0], end="")

## Cleanup

In [ ]:
collection_e.delete_all()
collection_c.delete_all

### To remove the very collections, run:
# collection_e.drop()
# collection_c.drop()